In [15]:
import open3d as o3d
import numpy as np
import math
from scipy.spatial import Delaunay

In [16]:
pcd = o3d.io.read_point_cloud("data/test_capture.ply")
axes = o3d.geometry.TriangleMesh.create_coordinate_frame()

In [17]:
plane_model, inliers = pcd.segment_plane(distance_threshold=0.01,
                                         ransac_n=3,
                                         num_iterations=10000)

[a, b, c, d] = plane_model
pcd = pcd.translate((0,0,d/c))
axes = o3d.geometry.TriangleMesh.create_coordinate_frame()

plane_pcd = pcd.select_by_index(inliers)
plane_pcd.paint_uniform_color([1.0, 0, 0])
stockpile_pcd = pcd.select_by_index(inliers, invert=True)
stockpile_pcd.paint_uniform_color([0, 0, 1.0])

PointCloud with 14867 points.

In [18]:
cos_theta = c / math.sqrt(a**2 + b**2 + c**2)
sin_theta = math.sqrt((a**2+b**2)/(a**2 + b**2 + c**2))
u_1 = b / math.sqrt(a**2 + b**2 )
u_2 = -a / math.sqrt(a**2 + b**2)

rotation_matrix = np.array([[cos_theta + u_1**2 * (1-cos_theta), u_1*u_2*(1-cos_theta), u_2*sin_theta],
                            [u_1*u_2*(1-cos_theta), cos_theta + u_2**2*(1- cos_theta), -u_1*sin_theta],
                            [-u_2*sin_theta, u_1*sin_theta, cos_theta]])

plane_pcd.rotate(rotation_matrix)
stockpile_pcd.rotate(rotation_matrix)
o3d.visualization.draw_geometries([plane_pcd, stockpile_pcd, axes])

In [19]:
xyz = np.asarray(pcd.points)
print(xyz.shape)
xy_to_z = {}
xy_catalog = set()
for point in xyz:
    xy = (point[0], point[1])
    xy_catalog.add(xy)
    xy_to_z[xy] = point[2]



(72198, 3)


In [8]:
radii = [0.005, 0.01, 0.02, 0.04, 0.08]
rec_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    stockpile_pcd, o3d.utility.DoubleVector(radii))
o3d.visualization.draw_geometries([rec_mesh])

In [9]:
hull, _ = stockpile_pcd.compute_convex_hull()
hull.orient_triangles()
hull_ls = o3d.geometry.LineSet.create_from_triangle_mesh(hull)
hull_ls.paint_uniform_color((1, 0, 0))
geo = o3d.visualization.draw_geometries([stockpile_pcd, hull_ls])
hull.get_volume()

0.009977564461581311

In [ ]:
vis = o3d.visualization.Visualizer()
vis.create_window(visible=False)
vis.add_geometry(geo)
vis.update_geometry(geo)
vis.poll_events()
vis.update_renderer()
vis.capture_screen_image("asd")
vis.destroy_window()